# K-Nearest Neighbor

In [1]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, roc_curve, auc, roc_auc_score

In [3]:
# Con la biblioteca pandas para manejo de datos, se cargan los datos del dataset Iris a la variable df_iris
df_train = pd.read_csv('df_train')
df_train.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24
0,-0.965194,-2.461538,-0.999170,1.638883,-0.012445,-0.550691,0.048277,1.037606,-1.532629,0.438425,...,-1.094361,0.612374,0.029113,-0.952970,0.883758,0.109869,-0.946612,-0.810787,1.017939,-0.193981
1,-2.010235,-0.583064,-1.564256,-0.462168,0.062850,1.835712,0.366935,0.268646,-1.625010,0.096149,...,0.203148,0.040519,-0.145090,0.215426,0.912894,0.701273,-0.731371,0.904574,1.588300,-1.629475
2,1.404573,0.830863,-0.260997,1.769054,-0.229199,-0.968585,-0.161798,1.310707,0.142251,0.676678,...,1.579726,1.150572,0.264154,-1.120993,3.062854,-1.529557,0.544768,-0.984611,0.562766,0.133422
3,0.088887,2.129559,-0.169476,-0.265115,0.438840,-0.916818,-0.444979,0.798028,0.155412,1.107905,...,1.475205,1.144960,0.281259,-1.181394,3.190902,-1.560147,0.542083,-1.163786,0.703665,0.444385
4,-0.091995,-1.191615,2.388757,-3.036106,0.627776,-0.085924,2.392148,1.670022,0.651484,-0.605123,...,-0.241812,0.299267,-0.305142,-0.213043,0.351165,-0.036503,0.749245,0.420417,-3.132510,-1.375841


In [4]:
# Con la biblioteca pandas para manejo de datos, se cargan los datos del dataset Iris a la variable df_iris
df_test = pd.read_csv('df_test')
df_test.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24
0,2.147853,1.666548,-0.086825,0.076311,1.088191,-0.186030,-0.478530,-0.058004,0.553014,1.241802,...,-1.541918,-0.471129,-0.104690,0.477865,-0.404092,0.914838,0.725396,0.634505,1.169001,-0.344828
1,-0.685470,1.752152,-1.730249,-0.172342,-0.279412,1.186319,1.394276,0.276136,0.446480,1.681607,...,1.013341,-0.569020,-0.309479,0.220008,-0.670621,0.374350,-0.395106,-1.084709,-1.927107,3.186851
2,-2.535785,-1.992615,-2.418738,-0.752230,2.924491,-3.724385,-3.114634,-0.187553,1.900389,-1.737932,...,1.775902,-0.608388,-0.430055,0.648788,-0.699955,0.735470,-1.679450,-0.735220,-1.018020,0.513048
3,-1.675202,1.610702,1.712692,-0.629990,-0.258625,-0.752654,-0.214901,0.313627,0.735308,1.365933,...,-0.772072,-1.080865,0.152700,0.492774,-0.297114,0.523644,0.073289,-0.490897,2.564693,-0.263620
4,2.409759,0.540428,-0.101821,0.276631,1.644325,0.370960,0.353518,-1.001202,-1.271318,-0.700280,...,0.109465,0.586679,0.121647,0.169950,-0.413875,0.843530,0.289632,1.133334,-0.417599,-0.193619


In [17]:
x_cols = df_train.columns
x_data = df_train[x_cols[:-1]]
x_data

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23
0,-0.965194,-2.461538,-0.999170,1.638883,-0.012445,-0.550691,0.048277,1.037606,-1.532629,0.438425,...,1.965340,-1.094361,0.612374,0.029113,-0.952970,0.883758,0.109869,-0.946612,-0.810787,1.017939
1,-2.010235,-0.583064,-1.564256,-0.462168,0.062850,1.835712,0.366935,0.268646,-1.625010,0.096149,...,-0.419713,0.203148,0.040519,-0.145090,0.215426,0.912894,0.701273,-0.731371,0.904574,1.588300
2,1.404573,0.830863,-0.260997,1.769054,-0.229199,-0.968585,-0.161798,1.310707,0.142251,0.676678,...,-0.352532,1.579726,1.150572,0.264154,-1.120993,3.062854,-1.529557,0.544768,-0.984611,0.562766
3,0.088887,2.129559,-0.169476,-0.265115,0.438840,-0.916818,-0.444979,0.798028,0.155412,1.107905,...,-0.123377,1.475205,1.144960,0.281259,-1.181394,3.190902,-1.560147,0.542083,-1.163786,0.703665
4,-0.091995,-1.191615,2.388757,-3.036106,0.627776,-0.085924,2.392148,1.670022,0.651484,-0.605123,...,-0.334979,-0.241812,0.299267,-0.305142,-0.213043,0.351165,-0.036503,0.749245,0.420417,-3.132510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31927,0.121514,-0.788258,2.362585,-2.307769,0.535232,0.358884,1.166238,-0.500608,-1.155323,-1.477309,...,0.943496,3.577993,0.126555,0.191371,1.750803,0.037625,-0.980780,2.282761,1.450054,-1.277337
31928,-0.881629,1.843032,1.044007,-0.697057,-0.202884,-0.911007,-0.188942,0.008385,0.688306,0.827398,...,-1.703018,-1.326924,-0.642528,-0.007130,0.231234,-0.086932,0.352400,0.273563,0.137608,1.303916
31929,1.131248,-0.787321,0.211130,-1.354795,-2.065229,0.071411,-0.764691,-1.646545,0.131780,1.334480,...,0.029052,0.324323,0.088349,0.212222,-0.111587,0.414779,-0.197526,-0.235174,-0.165819,-0.128662
31930,2.238385,0.573219,-0.049798,0.374514,2.154106,0.491172,-0.130993,-0.275976,-0.428450,0.254824,...,-0.189400,-0.125245,0.115320,-0.061547,0.141334,-0.341969,0.585251,0.196950,0.110906,0.351029


In [19]:
y_data = df_train[x_cols[-1]]
y_data

0       -0.193981
1       -1.629475
2        0.133422
3        0.444385
4       -1.375841
           ...   
31927    0.180638
31928    0.063675
31929    0.398081
31930   -0.138000
31931    0.996005
Name: PC24, Length: 31932, dtype: float64

In [20]:

# Division del dataset (entrenamiento 70%, prueba 30%)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3 ,random_state=31)
print('Dimensiones de datos de entrenamiento')
print(str(x_train.shape) + ' ' + str(y_train.shape))
print('Dimensiones de datos de prueba')
print(str(x_test.shape) + ' ' + str(y_test.shape))

Dimensiones de datos de entrenamiento
(22352, 23) (22352,)
Dimensiones de datos de prueba
(9580, 23) (9580,)


In [21]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

In [23]:
# Creacion del modelo KNN
knn = KNeighborsClassifier()

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(knn, param_grid, cv=4)

# Ajustar el modelo con GridSearchCV
grid_search.fit(x_data, y_data)

# Obtener los mejores parámetros y el mejor estimador
print("Mejores parámetros:", grid_search.best_params_)
best_knn = grid_search.best_estimator_

ValueError: 
All the 112 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
112 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Angel\miniconda3\envs\main\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Angel\miniconda3\envs\main\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\Angel\miniconda3\envs\main\lib\site-packages\sklearn\neighbors\_classification.py", line 238, in fit
    return self._fit(X, y)
  File "C:\Users\Angel\miniconda3\envs\main\lib\site-packages\sklearn\neighbors\_base.py", line 499, in _fit
    check_classification_targets(y)
  File "C:\Users\Angel